In [2]:
import pandas as pd
from cassandra.cluster import Cluster

In [ ]:
clstr=Cluster()
session=clstr.connect()

In [4]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [5]:
# Check keyspaces
rows=session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [7]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [8]:
# Check keyspaces
rows=session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [9]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    SKU INT, 
    Name TEXT, 
    Description TEXT, 
    Warehouse_Num INT,
    PRIMARY KEY(SKU)
);
""")

In [10]:
# Insert records

# Create a dictionary with your data
data = {
    'SKU': [1, 2, 3, 4, 5],
    'Name': ['Vinay', 'Vijay', 'Rakesh', 'Akshay', 'Praharsha'],
    'Description': ['Samsung new model is in our storage', 'IPHONE will be available in next week', 'There is limited stock for Oneplus', 'Iphone 14 is recommended to buy this week', 'Realme needs to be updated to attract customers'],
    'Warehouse_Num': [10, 20, 10, 30, 20]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


   SKU       Name                                      Description  \
0    1      Vinay              Samsung new model is in our storage   
1    2      Vijay            IPHONE will be available in next week   
2    3     Rakesh               There is limited stock for Oneplus   
3    4     Akshay        Iphone 14 is recommended to buy this week   
4    5  Praharsha  Realme needs to be updated to attract customers   

   Warehouse_Num  
0             10  
1             20  
2             10  
3             30  
4             20  


In [11]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, Name = {row.Name}, Description = {row.Description}, Warehouse_Num = {row.Warehouse_Num}")


SKU = 1, Name = Vinay, Description = Samsung new model is in our storage, Warehouse_Num = 10
SKU = 2, Name = Vijay, Description = IPHONE will be available in next week, Warehouse_Num = 20
SKU = 3, Name = Rakesh, Description = There is limited stock for Oneplus, Warehouse_Num = 10
SKU = 4, Name = Akshay, Description = Iphone 14 is recommended to buy this week, Warehouse_Num = 30
SKU = 5, Name = Praharsha, Description = Realme needs to be updated to attract customers, Warehouse_Num = 20


In [14]:
# Insert records in the database inventory

for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES ({row[0]}, '{row[1]}', '{row[2]}', {row[3]});
        """
       )



        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES (1, Vinay, Samsung new model is in our storage, 10);
        

        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES (2, Vijay, IPHONE will be available in next week, 20);
        

        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES (3, Rakesh, There is limited stock for Oneplus, 10);
        

        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES (4, Akshay, Iphone 14 is recommended to buy this week, 30);
        

        INSERT INTO m14.inventory (SKU, Name, Description, Warehouse_Num)     
        VALUES (5, Praharsha, Realme needs to be updated to attract customers, 20);
        


In [15]:
# Create a Index for the Warehouse_num
session.execute("""CREATE INDEX ON m14.inventory(Warehouse_Num);""")

In [16]:
# query the records for warehosue_num=10
rows = session.execute("select (SKU, Name, Description) from m14.inventory where Warehouse_Num=10 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")

1, Vinay, Samsung new model is in our storage
3, Rakesh, There is limited stock for Oneplus
